# Classifying Traffic Signs with OpenCV

For this project we will be trying split the traffic sign dataset into 3 separate classes:
- Stop signs
- Red Circles
- Blue rectangles/squares 

### Imports

In [11]:
import cv2 as cv
import numpy as np
import os, os.path
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt

### Useful functions

In [57]:
# Convert image color model to hsv
def cvt_color_to_hsv(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Remove image noise with gaussian blur (better at preserving edges)
def remove_noise(img):
    img = cv.GaussianBlur(img, 3)

# Add contrast to image by histogram equalization
def add_contrast(img):
    img = cv.equalizeHist(img)

# Filter reds in image using a mask by converting colors different than red black
def filter_red(img):
    # lower mask (0-10)
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower_red = np.array([0,70,50])
    upper_red = np.array([10,255,255])
    mask0 = cv.inRange(img, lower_red, upper_red)  

    # upper mask (170-180)
    lower_red = np.array([170,70,50])
    upper_red = np.array([180,255,255])
    mask1 = cv.inRange(img, lower_red, upper_red)

    mask = mask0 | mask1

    return mask

# Filter blues in image using a mask by converting colors different than blue black
def filter_blue(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower_blue = np.array([100,90,80])
    upper_blue = np.array([130,255,170])
    mask = cv.inRange(img, lower_blue, upper_blue)  

    return mask

# Find the contour with the largest area in a gray scale image
def find_largest_contour(img_blurred):
    contours,hierarchy = cv.findContours(img_blurred,2,1)
    cnt = contours
    big_contour = []
    max = 0
    for i in cnt:
        area = cv.contourArea(i)
        if(area > max):
            max = area
            big_contour = i 

    final = cv.drawContours(img, big_contour, -1, (0,255,0), 3)
    return final, big_contour

## Blue squares

In [3]:
blue_data_dir='./Dataset/blue_squares'
img = cv.imread(os.path.join(blue_data_dir,'road123.png'))
#billateral filtering
imgWithMeanFilter = cv.bilateralFilter(img, 15, 75, 75)

#histogram equalization
R, G, B = cv.split(imgWithMeanFilter)
output1_R = cv.equalizeHist(R)
output1_G = cv.equalizeHist(G)
output1_B = cv.equalizeHist(B)
equ = cv.merge((output1_R, output1_G, output1_B))



cv.imshow("original", img)
cv.imshow("filtered", equ)
cv.waitKey(0)
cv.destroyAllWindows()


## Stop signs

In [7]:
stop_data_dir = './Dataset/stop_signs'
img = cv.imread(os.path.join(stop_data_dir, 'road62.png'))
cv.imshow("original", img)
img2 = filter_red(img)
cv.imshow("filtered", img2)
cv.waitKey(0)
cv.destroyAllWindows()

In [60]:
filelist = []
for root, dirs, files in os.walk('./Dataset'):
	for file in files:
		filelist.append(os.path.join(root,file))
  
prediction = {}
for file in filelist:
    approx = []
    img = cv.imread(file)
    gray_red_blurred = cv.blur(filter_red(img), (3, 3))
    gray_blue_blurred = cv.blur(filter_blue(img), (3, 3))
    
    cv.imshow("filtered and blurred red", gray_red_blurred)
    cv.imshow("filtered and blurred blue", gray_blue_blurred)
        
    if cv.countNonZero(gray_blue_blurred) != 0:
        final, contour = find_largest_contour(gray_blue_blurred)
    
    if cv.countNonZero(gray_red_blurred) != 0:
        final, contour = find_largest_contour(gray_red_blurred)
    
    if len(contour) != 0:
        peri = cv.arcLength(contour, True)
        approx = cv.approxPolyDP(contour, 0.01 * peri, True)
    
    if len(approx) == 4:
        prediction[file] = 'square'
    elif len(approx) == 8:
        prediction[file] = 'stop_sign'
    elif len(approx) >= 9:
        prediction[file] = 'red_circle'
    else:
        prediction[file] = 'unrecognised'
    
    
    cv.imshow('final', final)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
print(prediction)
    

    

{'./Dataset\\blue_squares\\road123.png': 'square', './Dataset\\blue_squares\\road125.png': 'red_circle', './Dataset\\blue_squares\\road127.png': 'red_circle', './Dataset\\blue_squares\\road143.png': 'square', './Dataset\\blue_squares\\road193.png': 'red_circle', './Dataset\\blue_squares\\road204.png': 'stop_sign', './Dataset\\red_circles\\road100.png': 'red_circle', './Dataset\\red_circles\\road102.png': 'red_circle', './Dataset\\red_circles\\road104.png': 'red_circle', './Dataset\\red_circles\\road105.png': 'red_circle', './Dataset\\red_circles\\road106.png': 'red_circle', './Dataset\\red_circles\\road111.png': 'red_circle', './Dataset\\stop_signs\\road52.png': 'stop_sign', './Dataset\\stop_signs\\road57.png': 'stop_sign', './Dataset\\stop_signs\\road58.png': 'stop_sign', './Dataset\\stop_signs\\road59.png': 'stop_sign', './Dataset\\stop_signs\\road60.png': 'stop_sign', './Dataset\\stop_signs\\road62.png': 'stop_sign'}


Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

## Red Circles

This is the section to detect red circles

After processing the image to gray values holding as the white value the red features a contour finder is used to detect an ellipse correctly. It can also be used in the case of stop signs to obtain hexagons.

In [4]:
red_circle_data_dir = './Dataset/stop_signs'
img = cv.imread(os.path.join(red_circle_data_dir, 'road57.png'))
cv.imshow("original", img)
gray_blurred = cv.blur(filter_red(img), (3, 3))
cv.imshow("filtered and blurred", gray_blurred)

final, contour = find_largest_contour(gray_blurred)

peri = cv.arcLength(contour, True)

approx = cv.approxPolyDP(contour, 0.01 * peri, True)

print(len(approx))

# if len = 8 it is a stop sign ?
# else if len > 10 it is a red circle

cv.imshow('final', final)
cv.waitKey(0)
cv.destroyAllWindows()

[[[106  41]]

 [[105  42]]

 [[104  43]]

 ...

 [[109  41]]

 [[108  41]]

 [[107  41]]]
8


### Finding a perfect circle

By using the HoughCircles function we are able to detect perfect circles in an image.

There are some cases where it fits very well but if the sign is slanted no decent circle can be found

In [20]:
circles =  cv.HoughCircles(gray_blurred, cv.HOUGH_GRADIENT, 1.5, 100)
output = img.copy()
# ensure at least some circles were found
print(circles)
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv.circle(output, (x, y), r, (0, 255, 0), 4)
        cv.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1) #center of circle
        # show the output image
    cv.imshow("output", np.hstack([img, output]))

cv.waitKey(0)
cv.destroyAllWindows()

[[[216.75    183.75    172.20001]]]


### Commented out cell

To start we need to collect all of the xml files and the corresponding png files so we can parse them.
Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

In [ ]:
# xml_files = [name for name in os.listdir('./Dataset/annotations')]
# validation_dictionary = {}

# for name in xml_files:
#     # Parse the xml
#     mytree = ET.parse('./Dataset/annotations/' + name)
    
#     # Get number from name
#     order = ''.join(i for i in name if i.isdigit())
    
#     # Get sign type
#     validation_dictionary[order] = mytree.getroot().find('object').find('name').text

# sign_types = []
# for entry in validation_dictionary:
#     if validation_dictionary[entry] not in sign_types:
#         sign_types.append(validation_dictionary[entry])
# print(sign_types)
# print(validation_dictionary)

NameError: name 'validation_dictionary' is not defined